In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [25]:
import json
import pandas as pd
import os
import re
import string
import pickle
from transformers import BertForSequenceClassification
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
from sklearn.model_selection import train_test_split

from vecsim_app.categories import CATEGORIES
from vecsim_app.data_utils import papers

DATA_PATH = "/home/jovyan/arxiv/arxiv-metadata-oai-snapshot.json"
YEAR_CUTOFF = 2012
YEAR_PATTERN = r"(19|20[0-9]{2})"

In [ ]:
df = pd.DataFrame(papers(data_path=DATA_PATH, year_cutoff=YEAR_CUTOFF, year_pattern=YEAR_PATTERN))
len(df)

In [27]:
df.head(3)

,id,title,year,authors,categories,abstract,text
0,0704.0304,The World as Evolving Information,2012,Carlos Gershenson,"cs.IT,cs.AI,math.IT,q-bio.PE",This paper discusses the benefits of describ...,The World as Evolving Information This paper...
1,0704.2744,Nahm transform and parabolic minimal Laplace t...,2012,Szilard Szabo,math.AG,We prove that Nahm transform for integrable ...,Nahm transform and parabolic minimal Laplace t...
2,0704.2768,Heat Equations and the Weighted $\bar\partial$...,2012,Andrew Raich,"math.AP,math.CV",The purpose of this article is to establish ...,Heat Equations and the Weighted $\bar\partial$...


In [28]:
df['text'] = df['title'] + ' ' + df['abstract']
# df['categories'] = df['categories'].apply(lambda x: x.split(','))

In [29]:
df.iloc[0].categories

'cs.IT,cs.AI,math.IT,q-bio.PE'

In [30]:
# Split into train and test
df_train, df_test = train_test_split(df, test_size=0.2)

In [31]:
def get_tokenizer(tokenizer_model):
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)
    return tokenize_function, tokenizer

tokenize_function, tokenizer = get_tokenizer('bert-base-uncased')

loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--bert-base-uncased/snapshots/5546055f03398095e385d7dc625e636cc8910bf2/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /home/jovyan/.cache/huggingface/hub/models--bert-base-uncased/snapshots/5546055f03398095e385d7dc625e636cc8910bf2/vocab.txt
l

In [32]:
mlb = MultiLabelBinarizer()
# mlb.fit([[(k,v) for k, v in CATEGORIES.items()]]) #df_train['categories'])
mlb.fit([list(CATEGORIES.keys())]) #df_train['categories'])
mlb.classes_[:20]

array(['astro-ph', 'astro-ph.CO', 'astro-ph.EP', 'astro-ph.GA',
       'astro-ph.HE', 'astro-ph.IM', 'astro-ph.SR', 'cond-mat.dis-nn',
       'cond-mat.mes-hall', 'cond-mat.mtrl-sci', 'cond-mat.other',
       'cond-mat.quant-gas', 'cond-mat.soft', 'cond-mat.stat-mech',
       'cond-mat.str-el', 'cond-mat.supr-con', 'cs.AI', 'cs.AR', 'cs.CC',
       'cs.CE'], dtype=object)

In [11]:
def preprocess_data(examples):
    # take a batch of texts
    text = examples["text"]

    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    encoded_categories = mlb.transform([c.split(',') for c in examples['categories']]).astype(float)

    encoding["labels"] = encoded_categories

    return encoding

In [33]:
df_train_hf = Dataset.from_pandas(df_train[['text', 'categories']])
tokenized_train = df_train_hf.map(preprocess_data, batched=True)

df_test_hf = Dataset.from_pandas(df_test[['text', 'categories']])
tokenized_test = df_test_hf.map(preprocess_data, batched=True)

  0%|          | 0/328 [00:00<?, ?ba/s]

/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN', 'eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN', 'econ.TH', 'eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN', 'eess.SY', 'solv-int'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN', 'econ.TH'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown clas

  0%|          | 0/82 [00:00<?, ?ba/s]

/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN', 'eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['cond-mat', 'econ.TH', 'eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.TH', 'eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN', 'eco

In [34]:
# Debugging - get inverse transform

print("Reversed", mlb.inverse_transform(np.asarray(tokenized_test[0]['labels']).reshape(1, -1)))
print("Original categories", tokenized_test[0]['categories'])

Reversed [('cond-mat.soft',)]
Original categories cond-mat.soft


In [35]:
!rm -r checkpoint
!mkdir checkpoint
with open('checkpoint/mlb.pkl', 'wb') as handle:
    pickle.dump(mlb, handle, protocol=pickle.HIGHEST_PROTOCOL)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Training multi label class model

In [36]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=len(mlb.classes_), 
    problem_type="multi_label_classification"
)

loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--bert-base-uncased/snapshots/5546055f03398095e385d7dc625e636cc8910bf2/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",


In [40]:
# Adaptation: https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb 
# 
batch_size = 256
metric_name = "f1"

args = TrainingArguments(
    f"paper-multilabel-finetuning",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)


# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [41]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [42]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, categories, __index_level_0__. If text, categories, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 327018
  Num Epochs = 5
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 6390


RuntimeError: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 0; 14.76 GiB total capacity; 13.62 GiB already allocated; 27.75 MiB free; 13.83 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
eval = trainer.evaluate()
eval

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, categories, __index_level_0__. If text, categories, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


{'eval_loss': 0.3657287657260895,
 'eval_f1': 0.0009587727708533078,
 'eval_roc_auc': 0.4993164433982,
 'eval_accuracy': 0.0,
 'eval_runtime': 7.4244,
 'eval_samples_per_second': 134.691,
 'eval_steps_per_second': 8.486,
 'epoch': 5.0}

In [ ]:
text = df['text'].iloc[5]

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k, v in encoding.items()}

outputs = trainer.model(**encoding)
logits = outputs.logits

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.1)] = 1

In [ ]:
print(text)
print(mlb.inverse_transform(predictions.reshape(1, -1)))

Suppression of growth by multiplicative white noise in a parametric
  resonant system   The author studied the growth of the amplitude in a Mathieu-like equation
with multiplicative white noise. The approximate value of the exponent at the
extremum on parametric resonance regions was obtained theoretically by
introducing the width of time interval, and the exponents were calculated
numerically by solving the stochastic differential equations by a symplectic
numerical method. The Mathieu-like equation contains a parameter $\alpha$ that
is determined by the intensity of noise and the strength of the coupling
between the variable and the noise. The value of $\alpha$ was restricted not to
be negative without loss of generality. It was shown that the exponent
decreases with $\alpha$, reaches a minimum and increases after that. It was
also found that the exponent as a function of $\alpha$ has only one minimum at
$\alpha \neq 0$ on parametric resonance regions of $\alpha = 0$. This minimum
va

In [ ]:
trainer.save_model(output_dir='./checkpoint')

Saving model checkpoint to ./checkpoint
Configuration saved in ./checkpoint/config.json
Model weights saved in ./checkpoint/pytorch_model.bin
tokenizer config file saved in ./checkpoint/tokenizer_config.json
Special tokens file saved in ./checkpoint/special_tokens_map.json
